In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F


import sys
import numpy as np

def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=True)

def conv_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        init.xavier_uniform_(m.weight, gain=np.sqrt(2))
        init.constant_(m.bias, 0)
    elif classname.find('BatchNorm') != -1:
        init.constant_(m.weight, 1)
        init.constant_(m.bias, 0)

class wide_basic(nn.Module):
    def __init__(self, in_planes, planes, dropout_rate, stride=1):
        super(wide_basic, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, padding=1, bias=True)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=True)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=True),
            )

    def forward(self, x):
        out = self.dropout(self.conv1(F.relu(self.bn1(x))))
        out = self.conv2(F.relu(self.bn2(out)))
        out += self.shortcut(x)

        return out

class Wide_ResNet(nn.Module):
    def __init__(self, depth, widen_factor, dropout_rate, num_classes):
        super(Wide_ResNet, self).__init__()
        self.in_planes = 16

        assert ((depth-4)%6 ==0), 'Wide-resnet depth should be 6n+4'
        n = (depth-4)/6
        k = widen_factor

        print('| Wide-Resnet %dx%d' %(depth, k))
        nStages = [16, 16*k, 32*k, 64*k]

        self.conv1 = conv3x3(1,nStages[0])#3
        self.layer1 = self._wide_layer(wide_basic, nStages[1], n, dropout_rate, stride=1)
        self.layer2 = self._wide_layer(wide_basic, nStages[2], n, dropout_rate, stride=2)
        self.layer3 = self._wide_layer(wide_basic, nStages[3], n, dropout_rate, stride=2)
        self.bn1 = nn.BatchNorm2d(nStages[3], momentum=0.9)
        self.linear = nn.Linear(nStages[3], num_classes)

    def _wide_layer(self, block, planes, num_blocks, dropout_rate, stride):
        strides = [stride] + [1]*(int(num_blocks)-1)
        layers = []

        for stride in strides:
            layers.append(block(self.in_planes, planes, dropout_rate, stride))
            self.in_planes = planes

        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.bn1(out))
        out = F.avg_pool2d(out, 4)#8
        out = out.view(out.size(0), -1)
        out = self.linear(out)

        return out

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class PreActBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(PreActBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)

        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = F.relu(self.bn1(x))
        shortcut = self.shortcut(out) if hasattr(self, 'shortcut') else x
        out = self.conv1(out)
        out = self.conv2(F.relu(self.bn2(out)))
        out += shortcut
        return out


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def PreActResNet18():
    return ResNet(PreActBlock, [2,2,2,2])

def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

# Instantiate the ResNet model
# model = ResNet(BasicBlock, [3, 4, 6, 3])

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch
from torch.utils.data import DataLoader
from torchvision import models, datasets, transforms
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
import random
import numpy as np
seed =42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [4]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)


100%|██████████| 170498071/170498071 [00:03<00:00, 56388582.88it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
transform1 = transforms.Compose([transforms.RandomRotation(degrees = 10),
                                transforms.RandomHorizontalFlip(0.5),
                                transforms.ToTensor()])

trainset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform1, download=True)
testset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform1, download=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [5]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
def pgd_attack(x, y, model, niter=10, eps=0.07,stepsize=8/255,random_init=True):
    """
    PGD attack.

    Arguments:
        model (nn.Module): The model to attack.
        images (torch.Tensor): Input images to be perturbed.
        labels (torch.Tensor): True labels for the input images.
        eps (float): Maximum perturbation (default: 8/255).
        alpha (float): Step size (default: 2/255).
        steps (int): Number of steps (default: 10).
        random_start (bool): Use random initialization of delta (default: True).
        targeted (bool): Perform a targeted attack (default: False).

    Returns:
        torch.Tensor: Adversarial images.

    """
    device = x.device
    x = x.clone().detach().to(device)
    y = torch.tensor([y])
    y = y.clone().detach().to(device)
    loss = nn.CrossEntropyLoss()
    adv_x = x.clone().detach()

    if random_init:
        # Starting at a uniformly random point
        adv_x = adv_x + torch.empty_like(adv_x).uniform_(-eps, eps)
        adv_x = torch.clamp(adv_x, min=0, max=1).detach()

    for i in range(niter):
        adv_x.requires_grad = True
        outputs = model(adv_x)
        cost = loss(outputs,y)

        # Update adversarial images
        grad = torch.autograd.grad(cost, adv_x, retain_graph=False, create_graph=False)[0]

        adv_x = adv_x.detach() + stepsize * grad.sign()
        delta = torch.clamp(adv_x - x, min=-eps, max=eps)
        adv_x = torch.clamp(x + delta, min=0, max=1).detach()

    return adv_x

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.models import resnet50

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



# Load your trained ResNet-50 model
resnet50_model = torch.load('/kaggle/input/resnet-file/RESNET18_CIFAR_70EPOCHS(1).pt')


# Function to perform x refinement
def xrefine(x, yi, model, l):
    x.requires_grad = True
    x = x.to(device)  # Move to GPU
    output = model(x)
    loss = output[0, yi]  # Target label loss
    model.zero_grad()
    loss.backward()

    with torch.no_grad():
        x_grad = x.grad.sign()
        refined_x = x - l * x_grad
    return refined_x

# Function to generate adversarial pNML labels
def generate_adversarial_pnml_labels(x,y, model, labels, l):
    adv_labels = []
    x = x.unsqueeze(0).to(device) # Add batch dimension and move to GPU
    for yi in labels:
        refined_x = xrefine(x, yi, model, l)
        refined_x = refined_x.to(device)  # Move to GPU
        adv_pgd_sample = pgd_attack(refined_x,yi,model)
        output = model(adv_pgd_sample)
        probability = torch.softmax(output, dim=1)[0, yi].item()
        adv_labels.append(probability)
    return adv_labels

# Calculate clean accuracy
correct = 0
total = 0

for i in range(len(testset)):
    #if i >= 100:
        #break
    x, y = testset[i]
    labels = list(range(10))  # MNIST labels

    # Generate adversarial pNML labels
    adv_pnml_labels = generate_adversarial_pnml_labels(x,y, resnet50_model, labels, l=0.02)

    predicted_label = adv_pnml_labels.index(max(adv_pnml_labels))
    if predicted_label == y:
        correct += 1

    total += 1

clean_accuracy = correct / total
print("Clean accuracy:", clean_accuracy)

In [ ]:
def fgsm_attack(data, epsilon, input_grad):
    perturbed_data = data + epsilon * input_grad.sign()
    perturbed_data = torch.clamp(perturbed_data, 0, 1)  # Clip to maintain pixel range [0, 1]
    return perturbed_data

def generate_adversarial_examples(model, data, target, epsilon):
    data = data.to(device).requires_grad_()
    target = torch.tensor([target]).to(device)
    output = model(data)
    criterion = nn.CrossEntropyLoss()
    loss = criterion(output, target)
    model.zero_grad()
    loss.backward()
    input_grad = data.grad.data
    perturbed_data = fgsm_attack(data, epsilon, input_grad)
    return perturbed_data

# FGSM

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.models import resnet50

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



# Load your trained ResNet-50 model
resnet50_model = torch.load('/kaggle/input/resnet-file/RESNET18_CIFAR_70EPOCHS(1).pt')


# Function to perform x refinement
def xrefine(x, yi, model, l):
    x.requires_grad = True
    x = x.to(device)  # Move to GPU
    output = model(x)
    loss = output[0, yi]  # Target label loss
    model.zero_grad()
    loss.backward()

    with torch.no_grad():
        x_grad = x.grad.sign()
        refined_x = x - l * x_grad
    return refined_x

# Function to generate adversarial pNML labels
def generate_adversarial_pnml_labels(x,y, model, labels, l):
    adv_labels = []
    x = x.unsqueeze(0).to(device) # Add batch dimension and move to GPU
    for yi in labels:
        refined_x = xrefine(x, yi, model, l)
        refined_x = refined_x.to(device)  # Move to GPU
        adv_pgd_sample = generate_adversarial_examples(model,refined_x,yi,0.03)
        output = model(refined_x)
        probability = torch.softmax(output, dim=1)[0, yi].item()
        adv_labels.append(probability)
    return adv_labels

# Calculate clean accuracy
correct = 0
total = 0

for i in range(len(testset)):
    if i >= 100:
        break
    x, y = testset[i]
    labels = list(range(10))  # MNIST labels

    # Generate adversarial pNML labels
    adv_pnml_labels = generate_adversarial_pnml_labels(x,y, resnet50_model, labels, l=0.1)

    predicted_label = adv_pnml_labels.index(max(adv_pnml_labels))
    if predicted_label == y:
        correct += 1

    total += 1

clean_accuracy = correct / total
print("Clean accuracy:", clean_accuracy)

In [ ]:
# PGD attack function
def pgd_attack(x, y, model, niter=10, eps=0.1, stepsize=0.01, random_init=True):
    device = x.device
    x = x.clone().detach().to(device)
    y = torch.tensor([y]).to(device)  # Convert label y to tensor and move to device
    loss = nn.CrossEntropyLoss()
    adv_x = x.clone().detach()

    if random_init:
        adv_x = adv_x + eps * torch.randn_like(adv_x).to(device).detach().sign()
        adv_x = torch.clamp(adv_x, min=0, max=1)

    for _ in range(niter):
        adv_x.requires_grad = True
        output = model(adv_x)
        model.zero_grad()
        adv_loss = loss(output, y)
        adv_loss.backward()

        # PGD step
        adv_x = adv_x + stepsize * adv_x.grad.detach().sign()
        eta = torch.clamp(adv_x - x, min=-eps, max=eps)
        adv_x = torch.clamp(x + eta, min=0, max=1).detach()

    return adv_x

# Example usage:
# x is the clean image, y is the true label, model is your ResNet-50 model
# niter is the number of PGD iterations, eps is the epsilon value, stepsize is the step size for PGD attack

# adv_pgd_sample = pgd_attack(x, y, resnet50_model, niter=20, eps=0.1, stepsize=0.01, random_init=True)



In [ ]:
def xrefine(x, yi, model, l):
    x.requires_grad = True
    x = x.to(device)  # Move to GPU
    output = model(x)
    loss = output[0, yi]  # Target label loss
    model.zero_grad()
    loss.backward()

    with torch.no_grad():
        x_grad = x.grad.sign()
        refined_x = x - l * x_grad
    return refined_x

In [ ]:
def generate_adversarial_pnml_labels_with_pgd(x, model, labels, l, pgd_eps, pgd_iters, pgd_step):
    adv_labels = []
    x = x.unsqueeze(0).to(device)  # Add batch dimension and move to GPU
    for yi in labels:
        refined_x = xrefine(x, yi, model, l)
        refined_x = refined_x.to(device)  # Move to GPU
        
        # PGD attack on the refined sample
        adv_pgd_sample = pgd_attack(refined_x, yi, model, pgd_iters, pgd_eps, pgd_step)
        
        # Get the model output for the PGD-adversarial sample
        output = model(adv_pgd_sample)
        
        # Calculate probability for the specific label yi
        probability = torch.softmax(output, dim=1)[0, yi].item()
        adv_labels.append(probability)
    return adv_labels

In [ ]:
adv_correct = 0
total = 0
resnet50_model = torch.load('/kaggle/input/resnett/Resnet50_Mnist_10_Adv2_pt (1)')
resnet50_model.to(device)
resnet50_model.eval()


for i in range(len(testset)):
    if i >= 100:
        break
    x, y = testset[i]
    labels = list(range(10))  # MNIST labels
    # Generate adversarial pNML labels
    adv_pnml_labels = generate_adversarial_pnml_labels_with_pgd(x, resnet50_model, labels, l=0.1, pgd_eps=0.3, pgd_iters=10, pgd_step=2/255)

    # Get the predicted label based on the adversarial pNML labels
    predicted_label = adv_pnml_labels.index(max(adv_pnml_labels))
    if predicted_label == y:
        adv_correct += 1

    total += 1

adversarial_accuracy = adv_correct / total
print("Adversarial accuracy of pNML:", adversarial_accuracy)